* [Online CRC Calculator](https://crccalc.com/)
* Wikipedia [Cyclic redundancy check](https://en.wikipedia.org/wiki/Cyclic_redundancy_check)

In [181]:
def CRCTable(n, poly, right):
    result = list()
    for i in range(0, 256):
        temp  = i if right else i << 24
        for j in range(0, 8):
            if right:
                if temp & 0x01:
                    temp = (temp >> 1) ^ poly
                else:
                    temp = temp >> 1
            else:
                if temp & 0x80000000:
                    temp = ((temp << 1) % (1 << 32)) ^ poly
                else:
                    temp = (temp << 1) % (1 << 32)
        result.append(temp)
    return result

def CRCUpdate(data, crc, n, poly, right):
    assert type(data) == bytes
    table = CRCTable(n, poly, right)
    result = crc
    for byte in data:
        if right:
            result = (result >> 8) ^ table[(result & 0xff) ^ byte]
        else:
            result = ((result << 8) % (1 << 32)) ^ table[((result & 0xff000000) >> 24) ^ byte]
    return result

def CRC(data, n=32, poly=0xedb88320, init=0xffffffff, fin=0xffffffff, right=True):
    assert type(data) == bytes
    if type(poly) == list:
        temp = 0
        for term in poly:
            temp += 1 << (31 - term) if right else 1 << term
        poly = temp
    #print(hex(poly))
    result = CRCUpdate(data, init, n, poly, right)
    return result ^ fin